In [11]:
import os
import sys
import subprocess
import time
import re
import glob
import shutil
from google.colab import drive

# ==========================================
# UI & HELPER FUNCTIONS
# ==========================================
def ui_header(msg):
    print(f"\n\n{'='*60}\n{msg.center(60)}\n{'='*60}")

def ui_ask(msg):
    print(f"\n👉 {msg} ", end="")
    sys.stdout.flush()
    return input()

def ui_done(msg):
    print(f"\n\n✨ {msg}")

def ui_fail(msg, advice="", is_user=False):
    title = "PROCESS HALTED" if is_user else "SYSTEM ERROR"
    print(f"\n\n{'!'*60}\n{title.center(60, '!')}\n{'!'*60}")
    print(f"\n❌ {msg}")
    if advice:
        print(f"\n💡 ADVICE:\n{advice}")
    print("\n" + "!"*60)
    raise KeyboardInterrupt

def install_dependencies():
    print("📦 Installing system dependencies...")
    subprocess.run(["sudo", "apt-get", "install", "-y", "libtcmalloc-minimal4", "libglu1-mesa", "ffmpeg"],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# ==========================================
# MAIN EXECUTION
# ==========================================
def main():
    ui_header("MOUNTING GOOGLE DRIVE")

    if os.path.exists('/content/drive'):
        print("✅ Google Drive is already mounted.")
    else:
        print("📡 Attempting to mount Google Drive...")
        try:
            drive.mount('/content/drive')
        except Exception as e:
            ui_fail(
                f"Google Drive mounting failed: {str(e)}",
                "1. Enable third-party cookies for 'googleusercontent.com'.\n"
                "2. Or, click the 'Folder' icon (left sidebar) and click 'Mount Drive' manually."
            )

    DRIVE_BASE = '/content/drive/MyDrive/blender'
    tar_path = os.path.join(DRIVE_BASE, 'blender_tars')
    blend_path = os.path.join(DRIVE_BASE, 'blend_files')

    # Ensure base structure exists
    os.makedirs(tar_path, exist_ok=True)
    os.makedirs(blend_path, exist_ok=True)

    # --- PRE-FLIGHT FILE CHECK ---
    tars = sorted([f for f in os.listdir(tar_path) if f.endswith('.tar.xz')])
    blends = sorted([f for f in os.listdir(blend_path) if f.endswith('.blend')])

    if not tars or not blends:
        error_msg = "Required files are missing from Google Drive."
        advice_parts = []
        if not tars: advice_parts.append(f"• ENGINE: Upload a Linux Blender .tar.xz to: {tar_path}")
        if not blends: advice_parts.append(f"• PROJECT: Upload your .blend files to: {blend_path}")
        ui_fail(error_msg, "\n".join(advice_parts))

    # --- FILE SELECTION ---
    ui_header("SELECT BLENDER VERSION")
    for i, f in enumerate(tars): print(f" [{i}] {f}")
    while True:
        res = ui_ask("Select Engine Index:")
        if res.isdigit() and int(res) in range(len(tars)):
            SELECTED_TAR = tars[int(res)]
            break
        print(f"❗ Invalid index. Choose 0-{len(tars)-1}")

    ui_header("SELECT PROJECT FILE")
    for i, f in enumerate(blends): print(f" [{i}] {f}")
    while True:
        res = ui_ask("Select Project Index:")
        if res.isdigit() and int(res) in range(len(blends)):
            CHOSEN_BLEND_NAME = blends[int(res)]
            break
        print(f"❗ Invalid index. Choose 0-{len(blends)-1}")

    # --- RENDER CONFIGURATION ---
    ui_header("RENDER SETTINGS")

    engine_in = ui_ask("Render Engine (1: CYCLES, 2: EEVEE) [Default 1 - Press ⏎]:")
    RENDER_ENGINE = "BLENDER_EEVEE" if engine_in == "2" else "CYCLES"

    res_in = ui_ask("Resolution % (e.g., 50, 100, 200) [Default 100 - Press ⏎]:").strip()
    RES_PERCENT = int(res_in) if res_in.isdigit() else 100

    while True:
        OUT_NAME = ui_ask("Output Folder Name:").strip()
        if not OUT_NAME:
            print("❗ Output name cannot be empty.")
            continue
        FINAL_DRIVE_PATH = os.path.join(DRIVE_BASE, 'outputs', OUT_NAME)
        if os.path.exists(FINAL_DRIVE_PATH):
            print(f"❗ Folder '{OUT_NAME}' already exists. Choose another name.")
        else: break

    frame_input = ui_ask("Frames (e.g. 1-10,13-20) [Default 1 - Press ⏎]:").strip() or "1"
    frame_list = []
    try:
        for part in frame_input.split(','):
            part = part.strip()
            if '-' in part:
                start, end = map(int, part.split('-'))
                frame_list.extend(range(start, end + 1))
            else: frame_list.append(int(part))
        frame_list = sorted(list(set(frame_list)))
    except: ui_fail("Invalid frame format.")

    SELECTED_DEV = "CUDA"
    if RENDER_ENGINE == "CYCLES":
        dev_in = ui_ask("Device (1: CUDA - Stable, 2: OptiX - Faster) [Default 1 - Press ⏎]:")
        SELECTED_DEV = "OPTIX" if dev_in == "2" else "CUDA"

    # --- PREPARING ENVIRONMENT ---
    ui_header("PREPARING ENVIRONMENT")
    install_dependencies()

    LOCAL_BLEND = os.path.join("/content", CHOSEN_BLEND_NAME)
    print(f"🚚 Copying {CHOSEN_BLEND_NAME} to local storage...")
    shutil.copy(os.path.join(blend_path, CHOSEN_BLEND_NAME), LOCAL_BLEND)

    folder_name = SELECTED_TAR.replace(".tar.xz", "")
    BLENDER_BIN = f"/content/{folder_name}/blender"
    if not os.path.exists(BLENDER_BIN):
        print(f"📂 Extracting Blender...")
        subprocess.run(["tar", "-xf", os.path.join(tar_path, SELECTED_TAR), "-C", "/content/"], check=True)

    if not os.path.exists(BLENDER_BIN):
        potential = glob.glob("/content/blender-*/blender")
        if potential: BLENDER_BIN = potential[0]

    gpu_setup_code = f"""
import bpy
bpy.context.scene.render.engine = '{RENDER_ENGINE}'
bpy.context.scene.render.resolution_percentage = {RES_PERCENT}
if '{RENDER_ENGINE}' == 'CYCLES':
    cprefs = bpy.context.preferences.addons['cycles'].preferences
    cprefs.get_devices()
    cprefs.compute_device_type = '{SELECTED_DEV}'
    for d in cprefs.devices:
        d.use = (d.type == '{SELECTED_DEV}' or d.type == 'CPU')
    bpy.context.scene.cycles.device = 'GPU'
"""
    with open('/content/gpu_setup.py', 'w') as f: f.write(gpu_setup_code)

    # --- STARTING RENDER ---
    ui_header(f"STARTING RENDER ({len(frame_list)} Frames)")

    render_errors = 0
    for idx, frame in enumerate(frame_list):
        print(f"\n🎬 Processing Frame {frame} ({idx+1}/{len(frame_list)})")
        for f in glob.glob("/content/frame*"):
            if os.path.isfile(f): os.remove(f)

        cmd = [BLENDER_BIN, "-b", LOCAL_BLEND, "-P", "/content/gpu_setup.py", "-o", "/content/frame####", "-f", str(frame)]
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env={**os.environ, "PYTHONUNBUFFERED": "1"})

        print("🚦 Initializing Scene...")
        saved_path = None
        for line in process.stdout:
            line = line.strip()
            match = re.search(r"Samples (\d+)/(\d+)", line)
            if match:
                curr, total = int(match.group(1)), int(match.group(2))
                percent = (curr/total)*100
                bar = "█" * int(percent//5) + "░" * (20 - int(percent//5))
                sys.stdout.write(f"\r📊 {bar} {percent:.1f}% | Sample {curr}/{total}")
                sys.stdout.flush()
            elif any(x in line for x in ["Synchronizing", "Loading", "BVH", "Mem:"]):
                sys.stdout.write(f"\r⚙️  {line[:70]}...")
                sys.stdout.flush()
            elif "Saved:" in line:
                path_match = re.search(r"Saved: '(.*)'", line)
                if path_match: saved_path = path_match.group(1)
                print(f"\n📢 {line}")

        process.wait()

        if not saved_path or not os.path.exists(saved_path):
            found_frames = glob.glob(f"/content/frame{frame:04d}*")
            if found_frames: saved_path = found_frames[0]

        if saved_path and os.path.exists(saved_path):
            if not os.path.exists(FINAL_DRIVE_PATH):
                os.makedirs(FINAL_DRIVE_PATH, exist_ok=True)

            ext = os.path.splitext(saved_path)[1]
            shutil.move(saved_path, os.path.join(FINAL_DRIVE_PATH, f"frame_{frame:04d}{ext}"))
            print(f"✅ Saved to Drive.")
        else:
            print(f"❌ Render failed.")
            render_errors += 1

    # --- CLEANUP & VIDEO GENERATION ---
    ui_header("CLEANUP & POST-PROCESSING")
    if os.path.exists(LOCAL_BLEND):
        print(f"🧹 Removing local project: {CHOSEN_BLEND_NAME}")
        os.remove(LOCAL_BLEND)
    if os.path.exists('/content/gpu_setup.py'): os.remove('/content/gpu_setup.py')

    rendered_files = sorted(glob.glob(os.path.join(FINAL_DRIVE_PATH, "frame_*.*")))

    if len(rendered_files) > 1:
        print(f"🎞️ Found {len(rendered_files)} rendered frames.")
        while True:
            make_video = ui_ask("Generate .mp4 video from frames? (Y/N):").strip().lower()
            if make_video in ['y', 'n']: break
            print("❗ Invalid input. Please enter 'Y' or 'N'.")

        if make_video == 'y':
            # Ask for FPS with a default of 24
            fps_in = ui_ask("Frames Per Second (e.g., 24, 30, 60) [Default 24 - Press ⏎]:").strip()
            fps = int(fps_in) if fps_in.isdigit() else 24

            print(f"🎬 Encoding video at {fps} FPS with FFmpeg...")
            ext = os.path.splitext(rendered_files[0])[1]
            video_output = os.path.join(FINAL_DRIVE_PATH, f"{OUT_NAME}_preview.mp4")

            vf_filter = "format=yuv420p"
            if ext.lower() == ".exr": vf_filter = "gamma=2.2,format=yuv420p"

            ffmpeg_cmd = [
                "ffmpeg", "-y", "-framerate", str(fps),
                "-pattern_type", "glob", "-i", os.path.join(FINAL_DRIVE_PATH, f"frame_*{ext}"),
                "-vf", vf_filter, "-c:v", "libx264", "-crf", "18", video_output
            ]

            result = subprocess.run(ffmpeg_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, text=True)
            if result.returncode == 0:
                print(f"✅ Video created: {os.path.basename(video_output)}")
            else: print(f"❌ Video encoding failed: {result.stderr}")

    ui_done(f"Job finished. Folder: {FINAL_DRIVE_PATH}")

if __name__ == "__main__":
    try: main()
    except KeyboardInterrupt: print("\n\n🛑 Process halted.")



                   MOUNTING GOOGLE DRIVE                    
✅ Google Drive is already mounted.


                   SELECT BLENDER VERSION                   
 [0] blender-4.5.7-linux-x64.tar.xz
 [1] blender-5.0.0-linux-x64.tar.xz

👉 Select Engine Index: 0


                    SELECT PROJECT FILE                     
 [0] bmw27_gpu.blend
 [1] flat-archiviz.blend
 [2] lone-monk_cycles_and_exposure-node_demo.blend

👉 Select Project Index: 0


                      RENDER SETTINGS                       

👉 Render Engine (1: CYCLES, 2: EEVEE) [Default 1 - Press ⏎]: 1

👉 Resolution % (e.g., 50, 100, 200) [Default 100 - Press ⏎]: 

👉 Output Folder Name: bmw_cycles_anim

👉 Frames (e.g. 1-10,13-20) [Default 1 - Press ⏎]: 1-5

👉 Device (1: CUDA - Stable, 2: OptiX - Faster) [Default 1 - Press ⏎]: 2


                   PREPARING ENVIRONMENT                    
📦 Installing system dependencies...
🚚 Copying bmw27_gpu.blend to local storage...


                 STARTING RENDER (5 Frames)       